In [3]:
import numpy as np
print(np.version.version)
import galois

2.3.5


This code is provided as supplementary material of the lecture Channel Coding - Graph Based Codes (CC-GBC)

This code illustrates

MAP/ML decoding of a random parity-check code on the BEC

In [2]:
# Code generation
# size of the matrix
m = 12
n = 24

# erasure probablity of BEC
epsilon = 0.4

In [11]:
while(1):
    # construct parity-check matrix
    H = np.random.randint(1, 2, (m,n))

    # for encoding, split matrix into two parts
    # We have a systematic code x = (u p), and we split the parity-check matrix
    # into two parts H = (H_u H_p)
    # The condition H*x^T = 0 reads then H_u*u^T + H_p*p^T = 0
    # which we can solve for p^T = inv(H_p)*H_u*u^T
    # This means that H_p must be invertible, which we check first
    
    Hu = H[1:m][1:m]
    Hp = H[1:m][(m+1):]
    
    # abort construction if H_p is of full rank
    if np.linalg.matrix_rank(galois.GF(Hp)) == m:
        break

TypeError: Argument 'order' must be an instance of <class 'int'>, not <class 'numpy.ndarray'>.

In [10]:
# Encoding and Decoding using the given code
# encode
k = n-m

# random information word
u = np.random.randint(0, 2, (1,k))

# parity bits
p = np.linalg.inv(galois.GF(Hp)) @ galois.GF(Hu) @ np.matrix.transpose(galois.GF(u))
p = np.linalg.inv(p.view(np.ndarray))

# codeword
x = [u, p]

# verify if codeword
if np.mod(H @ x, 2) != 0:
    print("Not a codeword")

TypeError: Argument 'order' must be an instance of <class 'int'>, not <class 'numpy.ndarray'>.

In [ ]:
# transmission over BEC (-1 marks erasure)
y = x
with np.nditer(y, op_flags=(['readwrite'])) as it:
    for element in it:
        if np.random.randn() < epsilon:
            element[...] = -1

# erasure index set
E = np.argwhere(y == -1)

Ebar = setdiff(1:n, E);

% get sub-matrices
HE = H(:,E);
HEbar = H(:,Ebar);

% calculate s
s = mod(HEbar * y(Ebar)', 2);

In [ ]:
% can we solve the system of equations?
r = rank(gf(HE));

if r == numel(E)  % full rank? 
    % if full rank, solve directly using Gaussian elimination
    xE = gflineq(HE, s);
    
    % fill missing positions in y
    y(E) = xE;
    
    fprintf('Decoding successful: Transmitted and decoded word:\n');
    x
    y
else
    fprintf('H_E is not of full rank  ... trying to generate the set of all compatible solutions. This will take some time for large n\n');
    xEposs = de2bi(0:(2^numel(E)-1),numel(E),'left-msb');
    idx_compat = find(sum(mod(HE*xEposs'+repmat(s,1,size(xEposs,1)),2),1) == 0);
    
    % all compatible solutions
    X_compat = xEposs(idx_compat,:);
    
    % as there are more than one compatible solution, carry out bitwise MAP
    % decoding
    % find positions that are equal in the set of compatible codewords
    idx_bitwise = all(~diff(X_compat),1);
    
    % replace erased positions with recovered bits
    xh_MAP = y;
    xh_MAP(E(idx_bitwise)) = X_compat(1,idx_bitwise);
    
    fprintf('Transmitted:\n');
    x
    fprintf('Received:\n');
    y
    fprintf('Bitwise MAP decoding result:\n');
    xh_MAP    
end
fprintf('----------------------------------------------------------\n');